In [1]:
!cat ./venv/share/doc/drake/VERSION.TXT

20210312074643 9155f7b806992aada841cb61671c3c5ef71c0634

In [2]:
import numpy as np

from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import MultibodyPlant
from pydrake.multibody.tree import JacobianWrtVariable
from pydrake.manipulation.planner import (
    DifferentialInverseKinematicsParameters,
    DoDifferentialInverseKinematics,
)

In [3]:
def print_q_with_limits(plant, q, *, warn_threshold=0.1):
    q_lower = plant.GetPositionLowerLimits()
    q_upper = plant.GetPositionUpperLimits()
    assert len(q_lower) == len(q)
    name_width = 4
    precision = 5
    fmt = f"< 10.3f"
    for i, q_i in enumerate(q):
        name = f"q[{i}]:"
        q_i_lower = q_lower[i]
        q_i_upper = q_upper[i]
        dist = min(np.abs(q_i - q_i_lower), np.abs(q_i - q_i_upper))
        line = f"{name:<{name_width}} "
        line += f"{q_i_lower:{fmt}} <= {q_i:{fmt}} <= {q_i_upper:{fmt}}"
        line += f"   |   dist: {dist:{fmt}}"
        if dist < warn_threshold:
            line += " !!!"
        print(line)

In [4]:
def calc_desired_joint_velocities(
    plant, context, frame_E, *, dt, max_v, V_WEdes=None,
):
    ndof = plant.num_positions()
    assert plant.num_velocities() == ndof
    parameters = DifferentialInverseKinematicsParameters(
        num_positions=ndof, num_velocities=ndof
    )
    parameters.set_unconstrained_degrees_of_freedom_velocity_limit(0.0)
    parameters.set_timestep(dt)
    parameters.set_joint_velocity_limits((-max_v, max_v))
    result = DoDifferentialInverseKinematics(
        robot=plant,
        context=context,
        V_WE_desired=V_WEdes,
        frame_E=frame_E,
        parameters=parameters,
    )
    assert result.status == result.status.kSolutionFound, result.status
    return result.joint_velocities

In [5]:
plant = MultibodyPlant(time_step=0.01)
model_file = FindResourceOrThrow("drake/manipulation/models/franka_description/urdf/panda_arm.urdf")
arm = Parser(plant).AddModelFromFile(model_file)
frame_W = plant.world_frame()
frame_B = plant.GetFrameByName("panda_link0")
frame_G = plant.GetFrameByName("panda_link8")

plant.WeldFrames(frame_W, frame_B)
plant.Finalize()

context = plant.CreateDefaultContext()

# Functions w/ nasty globals.

def calc_Jv_WG():
    global plant, context
    return plant.CalcJacobianSpatialVelocity(
        context,
        with_respect_to=JacobianWrtVariable.kV,
        frame_B=frame_G,
        p_BP=[0, 0, 0],
        frame_A=frame_W,
        frame_E=frame_W,
    )

def calc_v_with_diff_ik(V_WGdes):
    # This does not produce a fixed point for certain combos of (q, v)...
    global plant, context, frame_G
    max_v = np.ones(7) * 0.5
    v = calc_desired_joint_velocities(
        plant,
        context,
        frame_E=frame_G,
        dt=0.1,
        max_v=max_v,
        V_WEdes=V_WGdes,
    )
    return v

def calc_v_simple(V_WGdes):
    # This can produce a fixed point.
    Jv_WG = calc_Jv_WG()
    v = np.linalg.lstsq(Jv_WG, V_WGdes)[0]
    return v

def attempt_fixed_point(v, recalc_v):
    global plant, context
    Jv_WG = calc_Jv_WG()
    V_WGdes = Jv_WG @ v
    v = recalc_v(V_WGdes)
    return v, V_WGdes

In [6]:
np.set_printoptions(formatter={"float_kind": lambda x: f"{x:,.3g}"})

q = np.array([0.041, -0.715, -0.0918, -2.79, 0.0112, 2.90, -0.80])
plant.SetPositions(context, arm, q)
v0 = np.zeros(7)

print_q_with_limits(plant, q)
print(f"v[0]: {v0}")

num_repeat = 5

print()
print("[ Use diff IK ]")
v = v0
for i in range(num_repeat):
    v_prev = v
    v, V_WGdes = attempt_fixed_point(v, calc_v_with_diff_ik)
    print(f"V_WGdes[{i}]: {V_WGdes}")
    print(f"v[{i+1}]:       {v}")
    print(f"Δv[{i+1}]:      {v - v_prev}")
    print("---")

print()
print("[ Use lstsq ]")
v = v0
for i in range(1, num_repeat):
    v_prev = v
    v, V_WGdes = attempt_fixed_point(v, calc_v_simple)
    print(f"V_WGdes[{i}]: {V_WGdes}")
    print(f"v[{i+1}]:       {v}")
    print(f"Δv[{i+1}]:      {v - v_prev}")
    print("---")

print()

q[0]: -2.897     <=  0.041     <=  2.897       |   dist:  2.856    
q[1]: -1.763     <= -0.715     <=  1.763       |   dist:  1.048    
q[2]: -2.897     <= -0.092     <=  2.897       |   dist:  2.805    
q[3]: -3.072     <= -2.790     <= -0.070       |   dist:  0.282    
q[4]: -2.897     <=  0.011     <=  2.897       |   dist:  2.886    
q[5]: -0.018     <=  2.900     <=  3.752       |   dist:  0.853    
q[6]: -2.897     <= -0.800     <=  2.897       |   dist:  2.097    
v[0]: [0 0 0 0 0 0 0]

[ Use diff IK ]
V_WGdes[0]: [0 0 0 0 0 0]
v[1]:       [-2.59e-23 -1.7e-22 6.29e-23 -2.57e-23 1.07e-23 -2e-22 4.96e-23]
Δv[1]:      [-2.59e-23 -1.7e-22 6.29e-23 -2.57e-23 1.07e-23 -2e-22 4.96e-23]
---
V_WGdes[1]: [1.9e-23 5.45e-23 -1.77e-24 -3.35e-23 1.6e-23 2.33e-23]
v[2]:       [-0.00188 -0.0141 0.00486 -0.00213 0.000218 -0.0166 0.00459]
Δv[2]:      [-0.00188 -0.0141 0.00486 -0.00213 0.000218 -0.0166 0.00459]
---
V_WGdes[2]: [0.00157 0.00452 -0.000147 -0.00278 0.00133 0.00193]
v[3]:       [-0.00